In [1]:
import os
from time import time
import yaml

from argparse import ArgumentParser, Namespace

from s3ts.api.nets.wrapper import WrapperModel
from s3ts.helper_functions import load_dm, str_time, get_parser

from pytorch_lightning import Trainer
from pytorch_lightning.utilities.model_summary import summarize

from s3ts.helper_functions import get_model

import torch
import numpy as np

import matplotlib.pyplot as plt

In [2]:
p = get_parser()

In [3]:
args = p.parse_args("--mode ts --dataset UCI-HAR --lr 0.001 --subjects_for_test 21 --window_size 32 --window_stride 2 --batch_size 128 --encoder_architecture cnn_gap --encoder_features 20 --decoder_architecture mlp --decoder_features 32 --decoder_layers 1 --pattern_size 32 --rho 0.1 --compute_n 300 --pattern_type syn --num_workers 8 --max_epochs 3 --normalize --reduce_imbalance --training_dir training_cv2 --n_val_subjects 4 --weight_decayL1 0 --weight_decayL2 0".split())

In [4]:
dm = load_dm(args)

class_changes = [0] + list(np.nonzero(np.diff(dm.stsds.SCS))[0])
print(len(class_changes), "number of class changes")

Loaded dataset UCI-HAR with a total of 815614 observations for window size 32
Sampling 59084 (balanced) observations per epoch.
Using 708997 observations for training, 106617 for validation and 25512 observations for test
1787 number of class changes


In [5]:
dm.ds_val.indices[0]

52569

In [6]:
np.unique(dm.stsds.SCS[dm.stsds.indices])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [7]:
dm.ds_val.indices

array([ 52569,  52570,  52571, ..., 579803, 579804, 579805])

In [8]:
dm.stsds.indices[52569]

74718

In [9]:
dm.stsds.SCS[74710:74719]

tensor([100, 100, 100, 100, 100, 100, 100, 100,   4])

In [14]:
dm.stsds[dm.ds_val.indices[0]]

74655 74719


(tensor([[ 0.5052,  0.5227,  0.4456,  0.5192,  0.4456,  0.4035,  0.3965,  0.4210,
           0.5893,  0.3229,  0.2808,  0.4210,  0.4351,  0.5192,  0.5192,  0.4140,
           0.3965,  0.6454,  0.5367,  0.5332,  0.5017,  0.6139,  0.4701,  0.2036,
           0.3755,  0.2703,  0.0879,  0.8313,  0.5437,  0.3088,  0.3755,  0.2668],
         [-0.8961, -0.9184, -0.8701, -0.7067, -0.6807, -0.7996, -0.8144, -0.9036,
          -0.6993, -0.6882, -0.5731, -0.6176, -0.7439, -0.7439, -0.7439, -0.5582,
          -0.4802, -0.4691, -0.4319, -0.4059, -0.5656, -0.3614, -0.5396, -0.4282,
          -0.4616, -0.4691, -0.5248, -0.8219, -0.8144, -0.7142, -0.7884, -0.8850],
         [-0.5556, -0.4824, -0.4207, -0.4515, -0.4631, -0.4361, -0.6057, -0.4593,
          -0.5132, -0.4246, -0.5016, -0.6134, -0.3976, -0.4284, -0.4284, -0.3205,
          -0.4708, -0.6828, -0.6404, -0.6866, -0.6558, -0.7829, -0.7020, -0.0315,
          -0.1433, -0.6673, -0.5402, -0.5171, -0.6982, -0.5826, -0.6558, -0.6018],
         [-0.

In [62]:
stride = 2
wsize = 5
a = 74718-wsize*stride+stride

In [60]:
print(dm.stsds.SCS[a:(74718+1):stride])
print(dm.stsds.SCS[a:(74718+1):stride].shape)

tensor([100, 100, 100, 100,   4])
torch.Size([5])


In [63]:
dm.stsds.STS[0, a:(74718+1):stride]

tensor([ 0.6314,  0.4631,  0.3755, -0.0208,  0.5893])

In [15]:
dm.stsds[dm.ds_val.indices[0]]

74655 74719


(tensor([[ 0.5052,  0.5227,  0.4456,  0.5192,  0.4456,  0.4035,  0.3965,  0.4210,
           0.5893,  0.3229,  0.2808,  0.4210,  0.4351,  0.5192,  0.5192,  0.4140,
           0.3965,  0.6454,  0.5367,  0.5332,  0.5017,  0.6139,  0.4701,  0.2036,
           0.3755,  0.2703,  0.0879,  0.8313,  0.5437,  0.3088,  0.3755,  0.2668],
         [-0.8961, -0.9184, -0.8701, -0.7067, -0.6807, -0.7996, -0.8144, -0.9036,
          -0.6993, -0.6882, -0.5731, -0.6176, -0.7439, -0.7439, -0.7439, -0.5582,
          -0.4802, -0.4691, -0.4319, -0.4059, -0.5656, -0.3614, -0.5396, -0.4282,
          -0.4616, -0.4691, -0.5248, -0.8219, -0.8144, -0.7142, -0.7884, -0.8850],
         [-0.5556, -0.4824, -0.4207, -0.4515, -0.4631, -0.4361, -0.6057, -0.4593,
          -0.5132, -0.4246, -0.5016, -0.6134, -0.3976, -0.4284, -0.4284, -0.3205,
          -0.4708, -0.6828, -0.6404, -0.6866, -0.6558, -0.7829, -0.7020, -0.0315,
          -0.1433, -0.6673, -0.5402, -0.5171, -0.6982, -0.5826, -0.6558, -0.6018],
         [-0.

In [52]:
def process_fft_full(STS): 
    # no tiene sentido hacer esto, ya que frecuencias que van a tener mayor peso son relacionadas con muchos sujetos y muchas clases a la vez
    # como la serie temporal es larga no funciona
    
    fft_size = 2**int(np.log2(STS.shape[1]))
    fft_short = np.fft.fft(STS, axis=-1, n=fft_size)
    fft_freq = np.fft.fftfreq(fft_size, d=1/50)

    return fft_short, fft_freq

In [53]:
out = process_fft_full(dm.stsds.STS)

In [54]:
sorted = np.argsort(np.abs(out[0]), axis=-1)
freq_sorted = out[1][sorted]

In [55]:
freq_sorted.shape

(6, 1048576)

In [49]:
1/1048576

9.5367431640625e-07

In [60]:
freq_sorted[:,-20:]

array([[ 4.29153442e-04, -4.29153442e-04, -1.62601471e-02,
         1.62601471e-02, -4.95910645e-03,  4.95910645e-03,
         2.86102295e-03, -2.86102295e-03, -1.71184540e-02,
         1.71184540e-02,  2.95639038e-03, -2.95639038e-03,
         2.38418579e-03, -2.38418579e-03,  3.00407410e-03,
        -3.00407410e-03,  2.57492065e-03, -2.57492065e-03,
         2.52723694e-03, -2.52723694e-03],
       [-2.71797180e-03,  2.71797180e-03, -4.95910645e-03,
         4.95910645e-03,  1.43051147e-04, -1.43051147e-04,
        -2.81333923e-03,  2.81333923e-03, -2.86102295e-03,
         2.86102295e-03, -2.67028809e-03,  2.67028809e-03,
         2.95639038e-03, -2.95639038e-03, -3.00407410e-03,
         3.00407410e-03, -2.57492065e-03,  2.57492065e-03,
        -2.52723694e-03,  2.52723694e-03],
       [-4.72068787e-03,  4.72068787e-03,  3.24249268e-03,
        -3.24249268e-03,  9.05990601e-04, -9.05990601e-04,
        -2.86102295e-03,  2.86102295e-03, -1.95503235e-03,
         1.95503235e-03, -6.6